In [1]:
import numpy as np
from matplotlib import pyplot as plt
import time

%matplotlib inline

# #!pip install ipympl
# %matplotlib widget
import sys
import os
sys.path.append(os.path.abspath("../external"))
sys.path.append(os.path.abspath("../external/causal_comp"))
sys.path.append(os.path.abspath("../external/tigramite"))

seed = 42
np.random.seed(seed)

# 2D model

## LKIFR

In [2]:
from causal_comp.function_liang_nvar import compute_liang_nvar

In [3]:
from causal_comp.function_liang_nvar import compute_sig

In [10]:
# Time series
load_series = True # True: load time series; False: compute and save time series
save_var = True # True: save Liang index and correlation coefficient (for plotting afterwards); False: don't save variables

filename = '../data/2D_series.npy'
var_filename = '../data/2D_liang.npy'
if load_series == True: # load time series
    t, X1, X2 = np.load(filename, allow_pickle=True)
    T, tau, R, error_T, error_tau, error_R, sig_T, sig_tau, sig_R = np.load(var_filename, allow_pickle=True)

In [11]:
# Time Parameters and Simulation Settings
dt = 0.001
tmax = 1000 # takes around 9 mins
nt = int(tmax / dt)
t = np.linspace(0, tmax, nt)
start_computation = int(10 / dt)  # discard transients
n_iter = 200 # bootstrap samples for Liang
nvar = 2

In [12]:
if load_series==False:
    # Simulate 2D Time Series
    a11, a12 = -1, 0.5
    a21, a22 = 0, -1
    sigma1, sigma2 = 0.1, 0.1
    dW1 = np.sqrt(dt) * np.random.normal(0, 1, nt)
    dW2 = np.sqrt(dt) * np.random.normal(0, 1, nt)
    X1, X2 = np.zeros(nt), np.zeros(nt)
    X1[0], X2[0] = 1.0, 2.0
    
    for i in range(nt - 1):
        X1[i + 1] = X1[i] + (a11 * X1[i] + a12 * X2[i]) * dt + sigma1 * dW1[i]
        X2[i + 1] = X2[i] + (a22 * X2[i] + a21 * X1[i]) * dt + sigma2 * dW2[i]
    
    # Apply Liang-Kleeman Info Flow on Post-Transient Segment
    print("Computing the Information Flow Rate...")
    start = time.time()
    xx = np.array([X1[start_computation:], X2[start_computation:]])
    T, tau, R, error_T, error_tau, error_R = compute_liang_nvar(xx, dt, n_iter)
    print(f"LKIFR computation time: {time.time() - start:.2f} seconds")

In [13]:
# Significance Check
conf = 1.96  # 95% confidence
sig_T = np.zeros((nvar, nvar))
sig_tau = np.zeros((nvar, nvar))
sig_R = np.zeros((nvar, nvar))

for j in range(nvar):
    for k in range(nvar):
        sig_T[j, k] = compute_sig(T[j, k], error_T[j, k], conf)
        sig_tau[j, k] = compute_sig(tau[j, k], error_tau[j, k], conf)
        sig_R[j, k] = compute_sig(R[j, k], error_R[j, k], conf)

# Results
print("\nLiang-Kleeman Info Transfer (T):")
print(T)
print("\nNormalized Info Transfer (% tau):")
print(tau)
print("\nPearson Correlation (R):")
print(R)

print("\nSignificance of T (1=significant):")
print(sig_T)
print("\nSignificance of tau:")
print(sig_tau)
print("\nSignificance of R:")
print(sig_R)


Liang-Kleeman Info Transfer (T):
[[-0.97944602  0.00578939]
 [ 0.10135035 -1.0512221 ]]

Normalized Info Transfer (% tau):
[[-49.96531961   0.27532049]
 [  5.17027232 -49.9919793 ]]

Pearson Correlation (R):
[[1.         0.22942272]
 [0.22942272 1.        ]]

Significance of T (1=significant):
[[1. 0.]
 [1. 1.]]

Significance of tau:
[[1. 0.]
 [1. 1.]]

Significance of R:
[[1. 1.]
 [1. 1.]]


In [ ]:
if load_series==False:
    # Save time series and results
    np.save('../data/2D_series_new.npy', [t, X1, X2])
    np.save('../data/2D_liang_new.npy', [T, tau, R, error_T, error_tau, error_R, sig_T, sig_tau, sig_R])

## PCMCI

In [14]:
from tigramite.data_processing import DataFrame
from tigramite.independence_tests.parcorr import ParCorr
from tigramite.pcmci import PCMCI

Tigramite takes input data as 2D Numpy array of shape (T:num of timesteps, N:num of variables)
Our simulated series X1, X2 are shaped (nt,) that is, each is a separate 1D timeseries. So, 
we will make the array (2, T) and then we\ll transpose it to be ready for Tigramite's DataFrame class.

In [15]:
data_raw = np.array([X1[start_computation:], X2[start_computation:]]).T  # shape: (T, 2)
dataframe = DataFrame(data_raw, var_names=["X1", "X2"])

# Set up ParCorr test and PCMCI object
parcorr = ParCorr(significance='analytic')
pcmci = PCMCI(dataframe=dataframe, cond_ind_test=parcorr)

# Run PCMCI (includes graph output by thresholding p_matrix)
results = pcmci.run_pcmci(tau_max=3, pc_alpha=0.05)

# FDR-corrected p-values
q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'], fdr_method='fdr_bh')
alpha_level = 0.05
significant = q_matrix < alpha_level

# Retrieve inferred parents from graph
parents_dict = pcmci.return_parents_dict(graph=results['graph'], val_matrix=results['val_matrix'])

# Show beta matrix at lag 1
print("\nPCMCI Path Coefficients (val_matrix) at lag=1:")
print(results['val_matrix'][:, :, 1])

print("\nInferred parents per variable (from graph):")
print(parents_dict)

print("\nFDR-corrected for significant p-values in q-matrix:")
print(significant)



PCMCI Path Coefficients (val_matrix) at lag=1:
[[0.70656187 0.00194443]
 [0.00071363 0.7062371 ]]

Inferred parents per variable (from graph):
{0: [(0, -1)], 1: [(1, -1)]}

FDR-corrected for significant p-values in q-matrix:
[[[False  True False False]
  [False False False False]]

 [[False False False False]
  [False  True False False]]]


# 6D model

In [19]:
from causal_comp.function_liang_nvar import compute_liang_nvar, compute_sig

In [ ]:
# Options
save_var = False # True: save Liang index and correlation coefficient (for plotting afterwards); False: don't save variables
load_series = True # True: load already saved time series (x,y,z); False: compute and save time series
nvar = 6 # number of variables (Liang [2021]: nvar=6)
n_iter = 400 # nuof bootstrap realizations (for computing the error in Liang index)
b_factor = 1 # amplitude of stochastic perturbation
conf = 1.96 # 1.96 if 95% confidence interval; 2.58 if 99% confidence interval; 1.65 if 90% confidence interval

# Time parameters
dt = 1 # time step
tmax = 1.e6 # total duration in unit times
nt = int(tmax / dt) # number of time steps (Liang [2021]: nt=10000)
t = np.linspace(0,tmax,nt) # time vector (varying between 0 and tmax with nt time steps)
start_computation = int(10000 / dt) # exclude the first transient times for computing correlation coefficient and rate of information transfer

# VAR parameters
alpha = np.array([0.1,0.7,0.5,0.2,0.8,0.3])
A = np.array([(0, 0, -0.6, 0, 0, 0),
			  (-0.5, 0, 0, 0, 0, 0.8),
			  (0, 0.7, 0, 0, 0, 0),
			  (0, 0, 0, 0.7, 0.4, 0),
			  (0, 0, 0, 0.2, 0, 0.7),
			  (0, 0, 0, 0, 0, -0.5)])
B = np.ones(nvar) * b_factor

# Random errors
mean_e = 0
std_e = 1
e = np.zeros((nvar,nt))
for var in np.arange(nvar):
    e[var,:] = np.random.normal(mean_e,std_e,nt)

# Initialization of variables
X = np.zeros((nvar,nt))
T = np.zeros((nvar,nvar))
tau = np.zeros((nvar,nvar))
R = np.zeros((nvar,nvar))
error_T = np.zeros((nvar,nvar))
error_tau = np.zeros((nvar,nvar))
error_R = np.zeros((nvar,nvar))

# Time series
filename = '../data/6D_series.npy'
if load_series == True: # load time series
    t,X[0,:],X[1,:],X[2,:],X[3,:],X[4,:],X[5,:] = np.load(filename,allow_pickle=True)

else: # compute time series and save them

    # VAR model
    for i in np.arange(nt-1):
        for var in np.arange(nvar):
            X[var,i+1] = alpha[var] + np.sum(A[var,:] * X[:,i]) + B[var] * e[var,i+1]

    # Save time series
    np.save(filename,[t, X[0,:], X[1,:], X[2,:], X[3,:], X[4,:], X[5,:]])

# Compute rate of information transfer and correlation coefficient using function_liang
xx = np.array((X[0,start_computation::],
               X[1,start_computation::],
               X[2,start_computation::],
               X[3,start_computation::],
               X[4,start_computation::],
               X[5,start_computation::]))
T,tau,R,error_T,error_tau,error_R = compute_liang_nvar(xx,dt,n_iter)

# Compute significance of rate of information transfer and correlation coefficient (by combining bootstrap samples)
sig_T = np.zeros((nvar,nvar))
sig_tau = np.zeros((nvar,nvar))
sig_R = np.zeros((nvar,nvar))
for j in np.arange(nvar):
    for k in np.arange(nvar):
        sig_T[j,k] = compute_sig(T[j,k],error_T[j,k],conf)
        sig_tau[j,k] = compute_sig(tau[j,k],error_tau[j,k],conf)
        sig_R[j,k] = compute_sig(R[j,k],error_R[j,k],conf)

In [ ]:
# Save results
if save_var == True:
    filename = '../data/6D_liang.npy'
    np.save(filename,[T,tau,R,error_T,error_tau,error_R,sig_T,sig_tau,sig_R])

In [27]:
# Display
print("\nLiang-Kleeman Info Transfer (T) [6D]:")
print(T)
print("\nNormalized Info Transfer (% tau) [6D]:")
print(tau)
print("\nPearson Correlation (R) [6D]:")
print(R)

print("\nSignificance of T [6D]:")
print(sig_T)
print("\nSignificance of tau [6D]:")
print(sig_tau)
print("\nSignificance of R [6D]:")
print(sig_R)



Liang-Kleeman Info Transfer (T) [6D]:
[[-9.99813539e-01  1.22778826e-02  9.45702946e-06 -1.99191622e-07
   1.47526177e-05  2.13753791e-05]
 [-1.21380506e-06 -1.00221419e+00 -9.15105787e-02 -1.20269742e-05
   5.23070109e-04 -7.74923041e-05]
 [-3.88334326e-02 -6.16887429e-05 -9.99346085e-01 -3.59360638e-05
   7.44331306e-05 -1.05505787e-04]
 [-9.43335953e-08  2.71796218e-05  3.12757633e-05 -2.99194938e-01
   3.70673050e-02 -3.99155694e-05]
 [-4.70135792e-05  3.91032358e-04  2.54368927e-04  4.57433271e-02
  -1.00229140e+00  1.67191010e-05]
 [-2.82393700e-05 -1.84482243e-01 -7.70080075e-05 -1.70598690e-05
  -1.85379191e-01 -1.49975035e+00]]

Normalized Info Transfer (% tau) [6D]:
[[-7.55675946e+01  8.62996499e-01  7.11341667e-04 -3.84588206e-05
   9.80053335e-04  1.13999479e-03]
 [-9.17414346e-05 -7.04443398e+01 -6.88327006e+00 -2.32210189e-03
   3.47488572e-02 -4.13283072e-03]
 [-2.93509638e+00 -4.33602200e-03 -7.51691124e+01 -6.93833713e-03
   4.94477926e-03 -5.62684981e-03]
 [-7.129892